In [ ]:
print(' Functions loaded!!!!')

In [1]:
def tolowercase(series):
    return series.str.lower()

In [ ]:
def strip_singlequote(series):
    return series.str.strip('\'')

In [ ]:
def timestamp2datetime(series):
    return  pd.to_datetime(series,unit='s',errors='coerce')

In [ ]:
def datetimestring2datetime(series):
    return  pd.to_datetime(series, format='%d-%m-%Y %H:%M',errors='coerce')

In [ ]:
def datetimeinvertedstring2datetime(series):
    return pd.to_datetime(series, format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [ ]:
def datetimeampm2datetime(series):
    return  pd.to_datetime(series,format='%d-%m-%Y %I:%M:%S %p',errors='coerce')

In [ ]:
def datetimemdyampm2datetime(series):
    return  pd.to_datetime(series, format='%m-%d-%Y %I:%M:%S %p',errors='coerce') 

In [1]:
def datetimestringparentesis2datetime(series):
    return  pd.to_datetime(series, format='%d/%m/%Y (%H:%M:%S',errors='coerce')

In [ ]:
def strip_parenthesisandsemicolon(series):
    return series.str.strip(');')

In [ ]:
def strip_insertintologs(series):
    return int(series.str.strip('INSERT INTO `logs` VALUES ('))

In [ ]:
def strip_tab(series):
    return series.str.strip('\t')

In [ ]:
def friendsenemies_type(series):
    return series.str.replace('f','friend').replace('e','enemy')

In [32]:
def friendsenemies_type2(series):
    return series.replace(1,'friend').replace(2,'enemy')

In [ ]:
def friendsenemies_type3(series):
    return series.replace(0,'friend').replace(1,'enemy')

In [5]:
def list_tables_with_insert(dumpfile):
    tables = []
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('insert'):
                table = re.findall(r'`(.*?)`', line)
                tables.append(table[0])
    return tables

In [ ]:
def get_keys_from_create_operation(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    keys=[]
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('create') and target_table in line:
                fast_forward = False
            if fast_forward:
                continue
            data = re.findall(r"\`(.*?)\`", line) #Step1: get the content between parentesis (i.e., insert operation)
            try: 
                keys.append(data[0])
            except Exception:
                pass
                
            if line.endswith(';'):
                break
    
    #for removing the name of the table and the (primary) key
    seen = set()
    seen_add = seen.add
    return [x for x in keys[1:] if not (x in seen or seen_add(x))]

# When there is NO more than 1 table with the same name 
###  faster because stops when find the end of the insert operation (;)

In [2]:
def read_inserted_table1(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
            if fast_forward:
                continue
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),', '', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                break
    sio.seek(0)
    return sio

# When there are more than 1 table with the same name

In [3]:
# For well formed SQL database dumps!
def read_inserted_table2(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

# Enriched version of read_inserted_table2, with the symbol and a list of tables 

In [32]:
def read_inserted_table3(dumpfile, target_table,symbol,tables):
    sio = StringIO()
    fast_forward = True
    already_header = False
    
    counter=1
    
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
                
            if line.lower().startswith('insert') and symbol+target_table+symbol in line:
                if counter in tables:
                    print('DEBUG (get table): '+line)
                    fast_forward = False
                else:
                    counter=counter+1
                    print('DEBUG (skip table)!')
                        
                if already_header:
                    continue
                
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                fast_forward = True
                already_header = True
                counter=counter+1
    sio.seek(0)
    return sio

# When there is NO more than 1 table with the same name (PROBLEM)

In [7]:
def read_inserted_table4(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
            if fast_forward:
                continue
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)                 
#                 newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                break
    sio.seek(0)
    return sio

# When the insert is NOT line by line but AN ENTIRE line

In [11]:
def read_inserted_table5(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert operation)
            
            #Main difference btw this fuction and the one before is this loop
            for record in data:
                try:
                    newline = record.strip('()') #Step2:remove parenthesis
                    newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                    newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                    newline=newline.replace('\'','') #Step5: remove single quotes
                    newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                    sio.write(newline)
                    sio.write("\n")
                except IndexError:
                    pass
                
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

In [1]:
def read_inserted_table_entirein1line(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert operation)
            
            #Main difference btw this fuction and the one before is this loop
            for record in data:
                try:
                    newline = record.strip('()') #Step2:remove parenthesis
                    newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                    newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                    newline=newline.replace('\'','') #Step5: remove single quotes
                    newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                    sio.write(newline)
                    sio.write("\n")
                except IndexError:
                    pass
                
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

# Enrichments

# * Libraries needed to retrieve information from external databases

In [7]:
import subprocess
import cfscrape
from lxml import etree
import os.path
import random
import time 

# * Function to enrich IP addresseswith AS information and country 

In [31]:
# THANKS TO: team-cymru.org
def iptoasn(iplist_teamcymruformat_filelocation):
    cat = subprocess.Popen(['cat', iplist_teamcymruformat_filelocation], 
                            stdout=subprocess.PIPE)
    
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43', '-q','1'],
                              stdin=cat.stdout,
                             stdout=subprocess.PIPE)
    


    # wait for the process to terminate
    out, err = netcat.communicate()
    errcode = netcat.returncode

    return out.decode()  

## * Function to check if an IP address was Tor node in a given moment

In [2]:
# 'date' MUST BE formated as: Year-month-day (2016-03-31)

# THANKS TO: exonerator.torproject.org
def WasTorNode(ip, date ):
    url="https://exonerator.torproject.org/?ip="+ip+"&timestamp="+date
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content    
    html_tree = etree.HTML(scraped_html)
    result=html_tree.xpath("//h3[@class='panel-title']/text()") # I was looking for <h3 class="panel-title">Result is positive</h3>
    tor_node=True if result == ['Result is positive'] else False
    return tor_node 


## * Relation between Attack dates and Login dates

In [ ]:
def nearestDate(base_date, date_list):
    nearest={}
    for date in date_list:
        if (base_date.timestamp() - date.timestamp())>=0:
            nearest[base_date.timestamp() - date.timestamp()]= date
    return nearest[min(nearest.keys())]